Data preprocessing  
https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html

In [ ]:
https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

In [1]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')

In [2]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [3]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [4]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)  # tensor of shape (n_words)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)  # the location where the next sample start, needed since torch.cat(text_list)
    text_list = torch.cat(text_list)  # of shape (the number of all words in the batch)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

model receives 338 integers, corresponding to words in a batch  
offsets required to cut each batch  
with offsets, each batch can have different sizes without padding

In [17]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()
        
    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [18]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for label, text in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [19]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [ ]:
from torch.utils.data.dataset import random_split
EPOCHS = 10 
LR = 5
BATCH_SIZE = 64

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train, split_valid = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS+1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

In [28]:
l, t, o = next(iter(dataloader))

In [32]:
t.shape

torch.Size([338])

In [7]:
import torch.nn as nn

In [14]:
>>> embedding_sum = nn.EmbeddingBag(10, 3, mode='max')
>>> # a batch of 2 samples of 4 indices each
>>> input = torch.LongTensor([1,2,4,5,4,3,2,9])
>>> offsets = torch.LongTensor([0,4])
>>> embedding_sum(input, offsets)

tensor([[ 0.9594,  1.8899, -0.0238],
        [ 0.9594,  2.8242,  2.1038]], grad_fn=<EmbeddingBagBackward>)

In [16]:
embedding_sum.weight

Parameter containing:
tensor([[-0.6480,  0.4361,  0.8501],
        [-1.4041,  1.8899, -0.8337],
        [-0.0420,  1.0114, -0.3521],
        [ 0.6951,  2.8242,  2.1038],
        [ 0.9594,  0.8461, -0.0238],
        [-1.5569,  1.5676, -0.7935],
        [-0.8019, -1.7431,  0.8489],
        [-0.2228, -0.0573,  0.2432],
        [-0.1542, -0.3362, -1.3588],
        [ 0.2763, -0.8879,  0.0159]], requires_grad=True)

In [23]:
embedding_sum(torch.LongTensor([1,2,4,5,4,3,2,9,1,2,3,4,5]), torch.LongTensor([0,4, 8]))

tensor([[ 5.8310, -3.0503, -1.9774],
        [ 1.9972, -0.4054, -3.2266],
        [ 5.4507, -2.8298, -3.3885]], grad_fn=<EmbeddingBagBackward>)

In [12]:
embedding = nn.Embedding(10, 3)

In [13]:
embedding(torch.LongTensor([1,2,4,5]))

tensor([[-0.0929,  0.6963,  1.1696],
        [-0.0305,  1.5499,  1.3172],
        [-1.0513, -1.0160, -2.4753],
        [ 0.0238, -0.3068, -1.4882]], grad_fn=<EmbeddingBackward>)

In [19]:
embedding(torch.LongTensor([4,3,2,9]))

tensor([[ 1.3412,  1.3789,  1.7488],
        [-0.7950,  0.0488,  0.2309],
        [ 0.3072,  1.1218,  1.0285],
        [-1.2522,  1.4263, -0.8899]], grad_fn=<EmbeddingBackward>)

In [15]:
embedding(input).sum(axis=0)

tensor([2.8646, 6.2018, 4.1726], grad_fn=<SumBackward1>)

In [7]:
offsets = [0, 100, 200, 300, 400, 500]

In [8]:
torch.tensor(offsets[:-1]).cumsum(dim=0)

tensor([   0,  100,  300,  600, 1000])